<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.4: Functional Programming
**Prev: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**<br>
**Next: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**

## Motivation
You saw functions in many previous modules, but now it's time to make our own and use them effectively.

## Setup

In [4]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /home/cricy/resources-ysyx/chisel-bootcamp/Main.scCompiling /home/cricy/resources-ysyx/chisel-bootcamp/Main.sc #2

path: String = "/home/cricy/resources-ysyx/chisel-bootcamp/source/load-ivy.sc"

This module uses the Chisel `FixedPoint` type, which currently resides in the experimental package.

In [5]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import chisel3.experimental._
import chisel3.internal.firrtl.KnownBinaryPoint

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import chisel3.experimental._

import chisel3.internal.firrtl.KnownBinaryPoint

---
# Functional Programming in Scala
Scala functions were introduced in Module 1, and you saw them being used a lot in the previous module. Here's a refresher on functions. Functions take any number of inputs and produce one output. Inputs are often called arguments to a function. To produce no output, return the `Unit` type. 

<span style="color:blue">**Example: Custom Functions**</span><br>
Below are some examples of functions in Scala.

In [4]:
// No inputs or outputs (two versions).
def hello1(): Unit = println("Hello!") // Unit represent that the function have not outputs.
def hello2 = println("Hello again!")

// Math operation: one input and one output.
def times2(x: Int): Int = 2 * x

// Inputs can have default values, and explicitly specifying the return type is optional.
// Note that we recommend specifying the return types to avoid surprises/bugs. NOTE!!!
def timesN(x: Int, n: Int = 2) = n * x  // NOTE: the return type is optional.

// Call the functions listed above.
hello1()
hello2
times2(4)
timesN(4)         // no need to specify n to use the default value
timesN(4, 3)      // argument order is the same as the order where the function was defined
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly

Hello!
Hello again!


defined function hello1
defined function hello2
defined function times2
defined function timesN
res3_6: Int = 8
res3_7: Int = 8
res3_8: Int = 12
res3_9: Int = 14

## Functions as Objects
Functions in Scala are first-class objects. That means we can assign a function to a `val` and pass it to classes, objects, or other functions as an argument.

<span style="color:blue">**Example: Function Objects**</span><br>
Below are the same functions implemented as functions and as objects.

In [6]:
// These are normal functions.
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// These are functions as vals.
// The first one explicitly specifies the return type.
val plus1val: Int => Int = x => x + 1 
val times2val = (x: Int) => x * 2

// Calling both looks the same.
plus1funct(4)
plus1val(4)
plus1funct(x=4)
//plus1val(x=4) // this doesn't work

defined function plus1funct
defined function times2funct
plus1val: Int => Int = ammonite.$sess.cmd5$Helper$$Lambda$2891/0x00000008014cbbb8@7b93c3f0
times2val: Int => Int = ammonite.$sess.cmd5$Helper$$Lambda$2892/0x00000008014d4000@17ecebc1
res5_4: Int = 5
res5_5: Int = 5
res5_6: Int = 5

Why would you want to create a `val` instead of a `def`? With a `val`, you can now pass the function around to other functions, as shown below. You can even create your own functions that accept other functions as arguments. Formally, functions that take or produce functions are called *higher-order functions*. You saw them used in the last module, but now you'll make your own!

<span style="color:blue">**Example: Higher-Order Functions**</span><br>
Here we show `map` again, and we also create a new function, `opN`, that accepts a function, `op`, as an argument.

In [7]:
// create our function
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)  // this uses the simple function and regard functions as parameter.
val myListTimes = myList.map(times2)

// create a custom function, which performs an operation on X N times using recursion
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)  // 7 + 1 + 1 + 1
opN(7, 3, times2) // 7 * 2 * 2 * 2

plus1: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2925/0x00000008014d7390@3464bfdd
times2: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2926/0x00000008014d7788@e57199d
myList: List[Int] = List(1, 2, 5, 9)
myListPlus: List[Int] = List(2, 3, 6, 10)
myListTimes: List[Int] = List(2, 4, 10, 18)
defined function opN
res6_6: Int = 10
res6_7: Int = 56

<span style="color:blue">**Example: Functions vs. Objects**</span><br>
A possibly confusing situation arises when using functions without arguments. Functions are evaluated every time they are called, while `val`s are evaluated at instantiation. 

In [8]:
import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function
val x = Random.nextInt
def y = Random.nextInt

// x was previously evaluated, so it is a constant
println(s"x = $x")
println(s"x = $x")

// y is a function and gets reevaluated at each call, thus these produce different results
println(s"y = $y")
println(s"y = $y")

x = -1043877968
x = -1043877968
y = -2057845416
y = -193908584


import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function

x: Int = -1043877968
defined function y

## Anonymous Functions
As the name implies, anonymous functions are nameless. There's no need to create a `val` for a function if we'll only use it once. 

<span style="color:blue">**Example: Anonymous Functions**</span><br>
The following example demonstrates this. They are often scoped (put in curly braces instead of parentheses). 

In [10]:
val myList = List(5, 6, 7, 8)

// add one to every item in the list using an anonymous function
// arguments get passed to the underscore variable
// these all do the same thing
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

// a common situation is to use case statements within an anonymous function   // NOTE: case statement more often
val myAnyList = List(1, 2, "3", 4L, myList)
myAnyList.map {
  case (_:Int|_:Long) => "Number"
  case _:String => "String"
  case _ => "other types"
}

myList: List[Int] = List(5, 6, 7, 8)
res9_1: List[Int] = List(6, 7, 8, 9)
res9_2: List[Int] = List(6, 7, 8, 9)
myAnyList: List[Any] = List(1, 2, "3", 4L, List(5, 6, 7, 8))
res9_4: List[String] = List(
  "Number",
  "Number",
  "String",
  "Number",
  "other types"
)

<span style="color:red">**Exercise: Sequence Manipulation**</span><br>
A common set of higher-order functions you'll use are `scanLeft`/`scanRight`, `reduceLeft`/`reduceRight`, and `foldLeft`/`foldRight`. It's important to understand how each one works and when to use them. The default directions for `scan`, `reduce`, and `fold` are left, though this is not guaranteed for all cases. 

In [12]:
val exList = List(1, 5, 7, 100)

// write a custom function to add two numbers, then use reduce to find the sum of all values in exList
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce{add}

// find the sum of exList using an anonymous function (hint: you've seen this before!)
val anon_sum = exList.reduce{_+_}

// find the moving average of exList from right to left using scan; make the result (ma2) a list of doubles
def avg(a: Int, b: Double): Double = (a + b) / 2.0
val ma2 = exList.foldLeft(0.0)(avg)

exList: List[Int] = List(1, 5, 7, 100)
defined function add
sum: Int = 113
anon_sum: Int = 113
defined function avg
ma2: List[Double] = List(8.875, 16.75, 28.5, 50.0, 0.0)

In [14]:
assert(add(88, 88) == 176)
assert(sum == 113)

assert(anon_sum == 113)

assert(avg(100, 100.0) == 100.0)
assert(ma2 == List(8.875, 16.75, 28.5, 50.0, 0.0))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

val anon\_sum = exList.reduce(\_ + \_)

def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)
</pre></article></div></section></div>

---
# Functional Programming in Chisel
Let's look at some examples of how to use functional programming when creating hardware generators in Chisel.

<span style="color:blue">**Example: FIR Filter**</span><br>
First, we'll revisit the FIR filter from previous examples. Instead of passing in the coefficients as parameters to the class or making them programmable, we'll pass a function to the FIR that defines how the window coefficients are calculated. This function will take the window length and bitwidth to produce a scaled list of coefficients. Here are two example windows. To avoid fractions, we'll scale the coefficients to be between the max and min integer values. For more on these windows, check out the [this Wikipedia page](https://en.wikipedia.org/wiki/Window_function).

In [18]:
// get some math functions
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window    below is the type of codffecient => val type = (the name of the first coefficient and the second cofficient) => {...
val TriangularWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window   // I convert the function as argument before to the normal function after like below
                    // these have the similar effects between function with function arguments and the normal function
def HammingWindow(length: Int, bitwidth: Int) : Seq[Int] = {  
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// check it out! first argument is the window length, and second argument is the bitwidth
TriangularWindow(10, 16)
HammingWindow(10, 16)

import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window    below is the type of codffecient => val type = (the name of the first coefficient and the second cofficient) => {...

TriangularWindow: (Int, Int) => Seq[Int] = ammonite.$sess.cmd17$Helper$$Lambda$5807/0x0000000801a33700@5ad15ff3
defined function HammingWindow
res17_3: Seq[Int] = Vector(
  0,
  14564,
  29127,
  43691,
  58254,
  58254,
  43691,
  29127,
  14564,
  0
)
res17_4: Seq[Int] = Vector(
  5243,
  12296,
  30155,
  50463,
  63718,
  63718,
  50463,
  30155,
  12296,
  5243
)

Now we'll create a FIR filter that accepts a window function as the argument. This allows us to define new windows later on and retain the same FIR generator. It also allows us to independently size the FIR, knowing the window will be recalculated for different lengths or bitwidths. Since we are choosing the window at compile time, these coefficients are fixed.

In [9]:
// our FIR has parameterized window length, IO bitwidth, and windowing function
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {            // ABOVE: window argument have input of function without actual parameters and convert to Seq[Int], 
                                      // so the Seq[Int] will pass to internal of class!!!
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy
  })

  // calculate the coefficients using the provided window function, mapping to UInts
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays
  // note: we avoid using a Vec here since we don't need dynamic indexing
  val delays = Seq.fill(length)(Wire(UInt(bitwidth.W))).scan(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults"
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs with bit growth
  val result = mults.reduce(_+&_)

  // connect output
  io.out := result
}

// the window argument actually gives the class a method to deal with the first Int arguments.
visualize(() => new MyFir(7, 12, HammingWindow))  
println(getVerilog(new MyFir(7, 12, HammingWindow)))  // NICE: the normal function and the function with function argument all can use in this class argument!!!

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: MyFir Pages: 1 -->
 
 
 MyFir 
 
 
 cluster_MyFir 
 
 MyFir 
 
<!-- cluster_MyFir_clock -->
 
 cluster_MyFir_clock 
 
 clock 
 
<!-- cluster_MyFir_reset -->
 
 cluster_MyFir_reset 
 
 reset 
 
<!-- cluster_MyFir_io_in -->
 
 cluster_MyFir_io_in 
 
 io_in 
 
<!-- struct_cluster_MyFir_REG -->
 
 struct_cluster_MyFir_REG 
 
 
 ● 
 
 REG 
 
 ● 
 
<!-- cluster_MyFir_io_in->struct_cluster_MyFir_REG -->
 
 cluster_MyFir_io_in->struct_cluster_MyFir_REG:in 
 
 
 
<!-- op_mul_55 -->
 
 op_mul_55 
 
 
 ● 
 
 mul 
 
 ● 
 
 328 
 
<!-- cluster_MyFir_io_in->op_mul_55 -->
 
 cluster_MyFir_io_in->op_mul_55:in1 
 
 
 
<!-- cluster_MyFir_io_out -->
 
 cluster_MyFir_io_out 
 
 io_out 
 
<!-- struct_cluster_MyFir_REG_1 -->
 
 struct_cluster_MyFir_REG_1 
 
 
 ● 
 
 REG_1 
 
 ● 
 
<!-- struct_cluster_MyFir_REG->struct_cluster_MyFir_REG_1 -->
 
 struct_cluster_MyFir_REG:out->struct_cluster_MyFir_REG_1:in 
 
 
 
<!-- op_mul_56 -->
 
 op_mul_56 
 
 
 ● 
 
 mul 
 
 ● 
 
 1270 
 
<!-- struct_cluster_MyFir_REG->op_mul_56 -->
 
 struct_cluster_MyFir_REG:out->op_mul_56:in1 
 
 
 
<!-- struct_cluster_MyFir_REG_2 -->
 
 struct_cluster_MyFir_REG_2 
 
 
 ● 
 
 REG_2 
 
 ● 
 
<!-- struct_cluster_MyFir_REG_1->struct_cluster_MyFir_REG_2 -->
 
 struct_cluster_MyFir_REG_1:out->struct_cluster_MyFir_REG_2:in 
 
 
 
<!-- op_mul_57 -->
 
 op_mul_57 
 
 
 ● 
 
 mul 
 
 ● 
 
 3154 
 
<!-- struct_cluster_MyFir_REG_1->op_mul_57 -->
 
 struct_cluster_MyFir_REG_1:out->op_mul_57:in1 
 
 
 
<!-- struct_cluster_MyFir_REG_3 -->
 
 struct_cluster_MyFir_REG_3 
 
 
 ● 
 
 REG_3 
 
 ● 
 
<!-- struct_cluster_MyFir_REG_2->struct_cluster_MyFir_REG_3 -->
 
 struct_cluster_MyFir_REG_2:out->struct_cluster_MyFir_REG_3:in 
 
 
 
<!-- op_mul_58 -->
 
 op_mul_58 
 
 
 ● 
 
 mul 
 
 ● 
 
 4096 
 
<!-- struct_cluster_MyFir_REG_2->op_mul_58 -->
 
 struct_cluster_MyFir_REG_2:out->op_mul_58:in1 
 
 
 
<!-- struct_cluster_MyFir_REG_4 -->
 
 struct_cluster_MyFir_REG_4 
 
 
 ● 
 
 REG_4 
 
 ● 
 
<!-- struct_cluster_MyFir_REG_3->struct_cluster_MyFir_REG_4 -->
 
 struct_cluster_MyFir_REG_3:out->struct_cluster_MyFir_REG_4:in 
 
 
 
<!-- op_mul_59 -->
 
 op_mul_59 
 
 
 ● 
 
 mul 
 
 ● 
 
 3154 
 
<!-- struct_cluster_MyFir_REG_3->op_mul_59 -->
 
 struct_cluster_MyFir_REG_3:out->op_mul_59:in1 
 
 
 
<!-- struct_cluster_MyFir_REG_5 -->
 
 struct_cluster_MyFir_REG_5 
 
 
 ● 
 
 REG_5 
 
 ● 
 
<!-- struct_cluster_MyFir_REG_4->struct_cluster_MyFir_REG_5 -->
 
 struct_cluster_MyFir_REG_4:out->struct_cluster_MyFir_REG_5:in 
 
 
 
<!-- op_mul_60 -->
 
 op_mul_60 
 
 
 ● 
 
 mul 
 
 ● 
 
 1270 
 
<!-- struct_cluster_MyFir_REG_4->op_mul_60 -->
 
 struct_cluster_MyFir_REG_4:out->op_mul_60:in1 
 
 
 
<!-- op_mul_61 -->
 
 op_mul_61 
 
 
 ● 
 
 mul 
 
 ● 
 
 328 
 
<!-- struct_cluster_MyFir_REG_5->op_mul_61 -->
 
 struct_cluster_MyFir_REG_5:out->op_mul_61:in1 
 
 
 
<!-- cluster_MyFir_mults_0 -->
 
 cluster_MyFir_mults_0 
 
 mults_0 
 
<!-- op_pad_63 -->
 
 op_pad_63 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- cluster_MyFir_mults_0->op_pad_63 -->
 
 cluster_MyFir_mults_0->op_pad_63:in1 
 
 
 
<!-- op_mul_55->cluster_MyFir_mults_0 -->
 
 op_mul_55:out->cluster_MyFir_mults_0 
 
 
 
<!-- cluster_MyFir_mults_1 -->
 
 cluster_MyFir_mults_1 
 
 mults_1 
 
<!-- op_add_62 -->
 
 op_add_62 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- cluster_MyFir_mults_1->op_add_62 -->
 
 cluster_MyFir_mults_1->op_add_62:in2 
 
 
 
<!-- op_mul_56->cluster_MyFir_mults_1 -->
 
 op_mul_56:out->cluster_MyFir_mults_1 
 
 
 
<!-- cluster_MyFir_mults_2 -->
 
 cluster_MyFir_mults_2 
 
 mults_2 
 
<!-- op_add_68 -->
 
 op_add_68 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- cluster_MyFir_mults_2->op_add_68 -->
 
 cluster_MyFir_mults_2->op_add_68:in2 
 
 
 
<!-- op_mul_57->cluster_MyFir_mults_2 -->
 
 op_mul_57:out->c

Elaborating design...
Done elaborating.
module MyFir(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [29:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
  reg [31:0] _RAND_3;
  reg [31:0] _RAND_4;
  reg [31:0] _RAND_5;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] REG; // @[cmd8.sc 14:20]
  reg [11:0] REG_1; // @[cmd8.sc 14:20]
  reg [11:0] REG_2; // @[cmd8.sc 14:20]
  reg [11:0] REG_3; // @[cmd8.sc 14:20]
  reg [11:0] REG_4; // @[cmd8.sc 14:20]
  reg [11:0] REG_5; // @[cmd8.sc 14:20]
  wire [20:0] mults_0 = io_in * 9'h148; // @[cmd8.sc 19:79]
  wire [22:0] mults_1 = REG * 11'h4f6; // @[cmd8.sc 19:79]
  wire [23:0] mults_2 = REG_1 * 12'hc52; // @[cmd8.sc 19:79]
  wire [24:0] mults_3 = REG_2 * 13'h1000; // @[cmd8.sc 19:79]
  wire [23:0] mults_4 = REG_3 * 12'hc52; // @[cmd8.sc 19:79]
  wire [22:0] mults_5 = REG_4 * 11'h4f6; // @[cmd8.sc 19:79]
  wire [20:0] mults_6 = REG_5 * 9'h148; // @[cmd8.sc 19:79]
  

defined class MyFir

Those last three lines could be easily combined into one. Also notice how we've handled bitwidth growth conservatively to avoid loss.

<span style="color:blue">**Example: FIR Filter Tester**</span><br>
Let's test our FIR! Previously, we provided a custom golden model. This time we'll use Breeze, a Scala library of useful linear algebra and signal processing functions, as a golden model for our FIR filter. The code below compares the Chisel output with the golden model output, and any errors cause the tester to fail.

Try uncommenting the print statment at the end just after the expect call. Also try changing the window from triangular to Hamming.

In [19]:
// math imports
import scala.math.{pow, sin, Pi}
import breeze.signal.{filter, OptOverhang}
import breeze.signal.support.{CanFilter, FIRKernel1D}
import breeze.linalg.DenseVector

// test parameters
val length = 7
val bitwidth = 12 // must be less than 15, otherwise Int can't represent the data, need BigInt
val window = HammingWindow   // here need the HammingWindow must being function with function argument!!!

// test our FIR
test(new MyFir(length, bitwidth, window)) { c =>
    
    // test data
    val n = 100 // input length
    val sine_freq = 10
    val samp_freq = 100

    // sample data, scale to between 0 and 2^bitwidth
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => (max_value/2 + max_value/2*sin(2*Pi*sine_freq/samp_freq*i)).toInt)
    println(s"input = ${sine.toArray.deep.mkString(", ")}")

    // coefficients
    val coeffs = window(length, bitwidth)
    println(s"coeffs = ${coeffs.toArray.deep.mkString(", ")}")

    // use breeze filter as golden model; need to reverse coefficients
    val expected = filter(
        DenseVector(sine.toArray),
        FIRKernel1D(DenseVector(coeffs.reverse.toArray), 1.0, ""),
        OptOverhang.None
    )
    expected.toArray // seems to be necessary
    println(s"exp_out = ${expected.toArray.deep.mkString(", ")}") // this seems to be necessary

    // push data through our FIR and check the result
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- 0 until n) {
        c.io.in.poke(sine(i).U)
        if (i >= length-1) { // wait for all registers to be initialized since we didn't zero-pad the data
            val expectValue = expected(i-length+1)
            // println(s"expected value is $expectValue")
            c.io.out.expect(expected(i-length+1).U)
            // println(s"cycle $i, got ${c.io.out.peek()}, expect ${expected(i-length+1)}")
        }
        c.clock.step(1)
    }
}

input = 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844, 2047, 3250, 3994, 3994, 3250, 2047, 844, 100, 100, 844
coeffs = 328, 1270, 3154, 4096, 3154, 1270, 328
exp_out = 46882438, 39608374, 27839200, 16070026, 8795962, 8795962, 16070026, 27839200, 39608374, 46882438, 46882438, 39608374, 27839200, 16070026, 8795962, 8795962, 16070026, 27839200, 39608374, 46882438, 46882438, 39608374, 27839200, 16070026, 8795962, 8795962, 16070026, 27839200, 39608374, 46882438, 46882438, 39608374, 27839200, 16070026, 8795962, 8795962, 16070026, 27839200, 396

import scala.math.{pow, sin, Pi}

import breeze.signal.{filter, OptOverhang}

import breeze.signal.support.{CanFilter, FIRKernel1D}

import breeze.linalg.DenseVector

// test parameters

length: Int = 7
bitwidth: Int = 12
window: (Int, Int) => Seq[Int] = ammonite.$sess.cmd17$Helper$$Lambda$5810/0x0000000801495800@7cb9d3ff

---
# Chisel Exercises
Complete the following exercises to practice writing functions, using them as arguments to hardware generators, and avoiding mutable data.

<span style="color:red">**Exercise: Neural Network Neuron**</span><br>
Our first example will have you build a neuron, the building block of fully-connected layers in artificial neural networks. Neurons take inputs and a set of weights, one per input, and produce one output. The weights and inputs are multiplied and added, and the result is fed through an activation function. In this exercise, you will implement different activation functions and pass them as an argument to your neuron generator.

![Neuron](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png)

First, complete the following code to create a neuron generator. The parameter `inputs` gives the number of inputs. The parameter `act` is a function that implements the logic of the activation function. We'll make the inputs and outputs 16-bit fixed point values with 8 fractional bits.

In [21]:
class Neuron(inputs: Int, act: FixedPoint => FixedPoint) extends Module {
  val io = IO(new Bundle {
    val in      = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val weights = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val out     = Output(FixedPoint(16.W, 8.BP))
  })
    val mac = io.in.zip(io.weights).map{ case(a: FixedPoint, b: FixedPoint) => a*b}.reduce(_+_)
    io.out := act(mac)
}
println(getVerilog(new Neuron(2,Step)))

Elaborating design...
Done elaborating.
module Neuron(
  input         clock,
  input         reset,
  input  [15:0] io_in_0,
  input  [15:0] io_in_1,
  input  [15:0] io_weights_0,
  input  [15:0] io_weights_1,
  output [15:0] io_out
);
  wire [31:0] _T = $signed(io_in_0) * $signed(io_weights_0); // @[cmd20.sc 7:81]
  wire [31:0] _T_1 = $signed(io_in_1) * $signed(io_weights_1); // @[cmd20.sc 7:81]
  wire [31:0] mac = $signed(_T) + $signed(_T_1); // @[cmd20.sc 7:93]
  wire [9:0] _T_5 = $signed(mac) <= 32'sh0 ? $signed(10'sh0) : $signed(10'sh100); // @[cmd19.sc 1:46]
  assign io_out = {{6{_T_5[9]}},_T_5}; // @[cmd19.sc 1:46]
endmodule



defined class Neuron

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val mac = io.in.zip(io.weights).map{ case(a:FixedPoint, b:FixedPoint) => a*b}.reduce(_+_)
  io.out := act(mac)
</pre></article></div></section></div>

Now let's create some activation functions! We'll use a threshold of zero. Typical activation functions are the sigmoid function and the rectified linear unit (ReLU).

The sigmoid we'll use is called the [logistic function](https://en.wikipedia.org/wiki/Logistic_function), given by 

$logistic(x) = \cfrac{1}{1+e^{-\beta x}}$

where $\beta$ is the slope factor. However, computing the exponential function in hardware is quite challenging and expensive. We'll approximate this as the step function.

$step(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             1  & \text{if } x \gt 0
       \end{cases}$

The second function, the ReLU, is given by a similar formula.

$relu(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             x  & \text{if } x \gt 0
       \end{cases}$

Implement these two functions below. You can specify a fixed-point literal like `-3.14.F(8.BP)`. 

In [20]:
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)

Step: FixedPoint => FixedPoint = ammonite.$sess.cmd19$Helper$$Lambda$5843/0x0000000801a70d08@24f625b7
ReLU: FixedPoint => FixedPoint = ammonite.$sess.cmd19$Helper$$Lambda$5844/0x0000000801a710e0@75d4029c

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)
</pre></article></div></section></div>

Finally, let's create a tester that checks the correctness of our Neuron. With the step activation function, neurons may be used as logic gate approximators. Proper selection of weights and bias can perform binary functions. We'll test our neuron using AND logic. Complete the following tester to check our neuron with the step function.

Note that since the circuit is purely combinational, the `reset(5)` and `step(1)` calls are not necessary.

In [22]:
// test our Neuron 
test(new Neuron(2, Step)) { c =>
    val inputs = Seq(Seq(-1, -1), Seq(-1, 1), Seq(1, -1), Seq(1, 1))

    // make this a sequence of two values
    val weights = Seq(1.0, 1.0)

    // push data through our Neuron and check the result (AND gate)
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- inputs) {
        c.io.in(0).poke(i(0).F(8.BP))
        c.io.in(1).poke(i(1).F(8.BP))
        c.io.weights(0).poke(weights(0).F(16.W, 8.BP))
        c.io.weights(1).poke(weights(1).F(16.W, 8.BP))
        c.io.out.expect((if (i(0) + i(1) > 0) 1 else 0).F(16.W, 8.BP))
        c.clock.step(1)
    }
    
}

test Neuron Success: 0 tests passed in 11 cycles in 0.010612 seconds 1036.54 Hz


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val weights  = Seq(1.0, 1.0)
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)